#　市区町村リストの取得

## 市区町村リストの取得

In [1]:
import json
import pandas as pd

with open('../cityList.json', 'r', encoding='utf-8') as file:
    cities = json.load(file)

df_cities = pd.DataFrame(cities)
df_cities

,prefCode,cityCode,cityName,bigCityFlag,prefName
0,01000,01100,札幌市,2,北海道
1,01000,01101,札幌市中央区,1,北海道
2,01000,01102,札幌市北区,1,北海道
3,01000,01103,札幌市東区,1,北海道
4,01000,01104,札幌市白石区,1,北海道
...,...,...,...,...,...
1917,47000,47361,久米島町,0,沖縄県
1918,47000,47362,八重瀬町,0,沖縄県
1919,47000,47375,多良間村,0,沖縄県
1920,47000,47381,竹富町,0,沖縄県


# Shapeの整形

In [29]:
# 港湾種別
port_class = [
    {"portClassCode": 11, "portClassName": "国際戦略港湾"},
    {"portClassCode": 12, "portClassName": "国際拠点港湾"},
    {"portClassCode": 13, "portClassName": "重要港湾"},
    {"portClassCode": 14, "portClassName": "地方港湾"},
    {"portClassCode": 15, "portClassName": "56条港湾"},
    {"portClassCode": 99, "portClassName": "その他"},
]

# 管理者区分
port_admin = [
    {"portAdminCode": 1, "portAdminName": "都道府県"},
    {"portAdminCode": 2, "portAdminName": "市区町村"},
    {"portAdminCode": 3, "portAdminName": "港務局"},
    {"portAdminCode": 4, "portAdminName": "一部事務組合"},
    {"portAdminCode": 5, "portAdminName": "港湾管理者なし"},
    {"portAdminCode": 0, "portAdminName": "その他"},
]

In [31]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd

# shapeファイルのパスを指定
shapefile_path = './shape/C02-14-g_PortAndHarbor.shp'

# GeoDataFrameを読み込む（エンコーディングを指定）
gdf = gpd.read_file(shapefile_path, encoding='shift-jis')  # または 'cp932' を試してみてください

# 必要な列名を指定
columns_to_keep = ['C02_002', 'C02_003', 'C02_004', 'C02_005', 'C02_006', 'C02_007', 'geometry']

# 指定した列のみを保持
gdf = gdf[columns_to_keep]

# 列名を変更
gdf = gdf.rename(columns={
    'C02_002': 'portType',
    'C02_003': 'cityCode',
    'C02_004': 'portCode',
    'C02_005': 'portName',
    'C02_006': 'adminType',
    'C02_007': 'adminName'
})

# portName列の値を修正（'元の値'+'港'の形式に変更）
gdf['portName'] = gdf['portName'].apply(lambda x: x + '港' if not x.endswith('港') else x)

# 港湾種別のマッピング辞書を作成
class_harbor_dict = {item['code']: item['name'] for item in class_harbor}

# 管理者区分のマッピング辞書を作成
admin_harbor_dict = {item['code']: item['name'] for item in admin_harbor}

# portTypeをマッピング
gdf['portType'] = gdf['portType'].map(class_harbor_dict).fillna('不明')

# adminTypeをマッピング
gdf['adminType'] = gdf['adminType'].map(admin_harbor_dict).fillna('不明')

# 属性情報をDataFrameとして取得（geometryを除く）
attr_df = pd.DataFrame(gdf.drop(columns='geometry'))

# DataFrameを表示
display(attr_df)

,portType,cityCode,portCode,portName,adminType,adminName
0,12,01205,01001,室蘭港,2,室蘭市
1,13,01202,01004,函館港,2,函館市
2,13,01203,01005,小樽港,2,小樽市
3,14,01345,01008,森港,2,森町
4,14,01361,01009,江差港,2,江差町
...,...,...,...,...,...,...
989,14,32201,32088,佐波港,2,松江市
990,14,32201,32087,才港,2,松江市
991,14,32201,32086,軽尾港,2,松江市
992,14,32201,32085,諸喰港,2,松江市


In [61]:
# gdfとdf_citiesをマージ
gdf = gdf.merge(df_cities, on='cityCode', how='left')

attr_df = pd.DataFrame(gdf.drop(columns='geometry'))
display(attr_df)

,cityCode,prefCode,cityName,bigCityFlag,prefName
0,37201,37000,高松市,0,香川県
1,40221,40000,太宰府市,0,福岡県
2,28382,28000,播磨町,0,兵庫県
3,27206,27000,泉大津市,0,大阪府
4,43104,43000,熊本市南区,1,熊本県
...,...,...,...,...,...
1458,01202,01000,函館市,0,北海道
1459,12219,12000,市原市,0,千葉県
1460,12219,12000,市原市,0,千葉県
1461,28201,28000,姫路市,0,兵庫県


# GeoJsonに変換して保存

In [72]:
import os

# GeoJSONファイルを保存するディレクトリを作成
output_dir = './geojson'
os.makedirs(output_dir, exist_ok=True)

# prefCodeの一覧を取得
pref_codes = gdf['prefCode'].unique()

for pref_code in pref_codes:
    # prefCodeでフィルタリング
    gdf_filtered = gdf[gdf['prefCode'] == pref_code]
    
    # ファイル名の生成（prefCodeを使用）
    filename = f"{pref_code}.geojson"
    output_path = os.path.join(output_dir, filename)
    
    # GeoJSONとして保存
    gdf_filtered.to_file(output_path, driver='GeoJSON')
    print(f"{filename} を保存しました。")


37000.geojson を保存しました。
40000.geojson を保存しました。
28000.geojson を保存しました。
27000.geojson を保存しました。
43000.geojson を保存しました。
12000.geojson を保存しました。
41000.geojson を保存しました。
34000.geojson を保存しました。
11000.geojson を保存しました。
09000.geojson を保存しました。
23000.geojson を保存しました。
14000.geojson を保存しました。
35000.geojson を保存しました。
01000.geojson を保存しました。
19000.geojson を保存しました。
08000.geojson を保存しました。
13000.geojson を保存しました。
22000.geojson を保存しました。
26000.geojson を保存しました。
29000.geojson を保存しました。
05000.geojson を保存しました。
33000.geojson を保存しました。
06000.geojson を保存しました。
15000.geojson を保存しました。
24000.geojson を保存しました。
31000.geojson を保存しました。
32000.geojson を保存しました。
03000.geojson を保存しました。
47000.geojson を保存しました。
38000.geojson を保存しました。
18000.geojson を保存しました。
16000.geojson を保存しました。
07000.geojson を保存しました。
02000.geojson を保存しました。
04000.geojson を保存しました。
25000.geojson を保存しました。
17000.geojson を保存しました。
46000.geojson を保存しました。
42000.geojson を保存しました。
45000.geojson を保存しました。
20000.geojson を保存しました。
10000.geojson を保存しました。
21000.geojson を保存しました。
44000.geojs

# TopoJsonに変換して保存

In [73]:

import subprocess

for pref_code in pref_codes:

    input_geojson = f"./geojson/{pref_code}.geojson"
    output_topojson = f"./topojson/{pref_code}.topojson"

    # mapshaper-npmを使用してGeoJSONをTopoJSONに変換
    command = f"mapshaper {input_geojson} -o format=topojson {output_topojson}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)


